In [20]:
import tensorflow as tf
tf.set_random_seed(1)
import cv2
from keras.models import Model
from keras.layers import Input, merge
from keras.optimizers import Adam
from fr_utils import *
from inception_blocks_v2 import *
import numpy as np
from tqdm import tqdm

In [2]:
from pprint import pprint
import matplotlib.pyplot as plt
from keras.utils import to_categorical

In [3]:


ALPHA = 0.2  # Triplet Loss Parameter

# Source: https://github.com/davidsandberg/facenet/blob/master/src/facenet.py
def triplet_loss(x):
    anchor, positive, negative = x

    pos_dist = tf.reduce_sum(tf.square(tf.subtract(anchor, positive)), 1)
    neg_dist = tf.reduce_sum(tf.square(tf.subtract(anchor, negative)), 1)

    basic_loss = tf.add(tf.subtract(pos_dist, neg_dist), ALPHA)
    loss = tf.reduce_mean(tf.maximum(basic_loss, 0.0), 0)

    return loss

# Builds an embedding for each example (i.e., positive, negative, anchor)
# Then calculates the triplet loss between their embedding.
# Then applies identity loss on the triplet loss value to minimize it on training.
def build_model(input_shape):
    # Standardizing the input shape order
    K.set_image_dim_ordering('th')

    positive_example = Input(shape=input_shape)
    negative_example = Input(shape=input_shape)
    anchor_example = Input(shape=input_shape)

    # Create Common network to share the weights along different examples (+/-/Anchor)
    embedding_network = faceRecoModel(input_shape)

    positive_embedding = embedding_network(positive_example)
    negative_embedding = embedding_network(negative_example)
    anchor_embedding = embedding_network(anchor_example)

    loss = merge([anchor_embedding, positive_embedding, negative_embedding],
                 mode=triplet_loss, output_shape=(1,))
    model = Model(inputs=[anchor_example, positive_example, negative_example],
                  outputs=loss)
    model.compile(loss='mean_absolute_error', optimizer=Adam())

    return model

# When fitting the model (i.e., model.fit()); use as an input [anchor_example,
# positive_example, negative_example] in that order and as an output zero.
# The reason to use the output as zero is that you are trying to minimize the 
# triplet loss as much as possible and the minimum value of the loss is zero.

In [4]:
fmodel = build_model(input_shape=(3, 96, 96))

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:36: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
/usr/local/lib/python3.5/dist-packages/keras/legacy/layers.py:458: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)


In [5]:
## sample dataset
# m = 5
# x  = []
# y = []
# for i in range(m):
#     a = np.random.random( (1, 3, 96, 96) )
#     b = np.random.random( (1, 3, 96, 96) )
#     c = np.random.random( (1, 3, 96, 96) )
#     x.append([a,b,c])
    
#     y.append(np.zeros((1,1)) )


In [6]:
x_data = np.load('mnist_triplet_dataset.npy')


In [7]:
def process_img(a_img, p_img, n_img):
    a_img = cv2.cvtColor(a_img,cv2.COLOR_GRAY2RGB) 
    p_img= cv2.cvtColor(p_img,cv2.COLOR_GRAY2RGB)
    n_img = cv2.cvtColor(n_img,cv2.COLOR_GRAY2RGB)

    a_img = cv2.resize(a_img, (96, 96)) / 255.0
    p_img = cv2.resize(p_img, (96, 96)) / 255.0
    n_img = cv2.resize(n_img, (96, 96)) / 255.0


    a_img = np.rollaxis(a_img, 2, 0)
    a_img = a_img.reshape((1, a_img.shape[0],  a_img.shape[1],  a_img.shape[2]))

    p_img = np.rollaxis(p_img, 2, 0)
    p_img = p_img.reshape((1, p_img.shape[0],  p_img.shape[1],  p_img.shape[2]))

    n_img = np.rollaxis(n_img, 2, 0)
    n_img = n_img.reshape((1, n_img.shape[0],  n_img.shape[1],  n_img.shape[2]))
    
    return a_img, p_img, n_img

In [21]:
for x in tqdm(x_data):
    a_img, p_img, n_img = process_img(x[0], x[1], x[2])
    
    verbose = 1 if i % 1000 == 0 else 0        
    fmodel.fit([a_img, p_img, n_img], np.zeros((1,1)), verbose=0)
    
#     if verbose == 1:
#         print('\nsteps ', i)
    
fmodel.save('mnist_weight.h5')
    

100%|██████████| 50000/50000 [1:01:02<00:00, 13.65it/s]


In [28]:
fmodel.summary()

# from keras.utils import plot_model
# plot_model(fmodel, to_file='model.png')

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_3 (InputLayer)             (None, 3, 96, 96)     0                                            
____________________________________________________________________________________________________
input_1 (InputLayer)             (None, 3, 96, 96)     0                                            
____________________________________________________________________________________________________
input_2 (InputLayer)             (None, 3, 96, 96)     0                                            
____________________________________________________________________________________________________
FaceRecoModel (Model)            (None, 128)           3743280     input_1[0][0]                    
                                                                   input_2[0][0]           

In [38]:
model_new = Model(inputs=fmodel.layers[3].get_input_at(-1), 
                  outputs=fmodel.layers[3].get_output_at(-1))
print(model_new.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 3, 96, 96)         0         
_________________________________________________________________
FaceRecoModel (Model)        (None, 128)               3743280   
Total params: 3,743,280
Trainable params: 3,733,968
Non-trainable params: 9,312
_________________________________________________________________
None


In [39]:
model_new.save('Siamese_Networks_weight.h5')

In [59]:
from keras.datasets import mnist
(_, _), (test_x, test_y) = mnist.load_data()

test_x = test_x[0:2000]
test_y = test_y[0:2000]

emb_tsv_str = '' 
emb_meta_str = ''

for index, img in enumerate (test_x):
    
    img, _, _ = process_img(img, img, img)
    
    res = model_new.predict(img)
    
    res_str ='\t'.join ([str(x) for x in res.tolist()[0]])
    emb_tsv_str += res_str + '\n'
    
    emb_meta_str += str(test_y[index]) + '\n'
    
    
with open('embd_mnist.tsv', 'w') as f:
    f.write(emb_tsv_str)
    
with open('emb_meta_str.tsv', 'w') as f:
    f.write(emb_meta_str)